In [ ]:
#imports
from collections import deque
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import serial
import threading
import queue
import datetime
import time
import tkinter as tk
from tkinter import messagebox
import re

In [ ]:
#Quick db setup
database_name = "accel2"
import mysql
import mysql.connector
server = mysql.connector.connect(
    host='127.0.0.1', #local host
    user='root',    # root user: has access to everything
    passwd='ICELABMR',
)
server_cursor = server.cursor()
drop_database_query = "DROP DATABASE IF EXISTS {}".format(database_name)
server_cursor.execute(drop_database_query)
server.commit()
server_cursor.execute("CREATE DATABASE {}".format(database_name))
db = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    passwd='ICELABMR',
    database=database_name #connect to newly created database
)
dbc = db.cursor()

In [ ]:
#quick table setup
table_1 = "Force_Data"
entry_1 = "`Force`"
entry_2 = "`Timestamp`"
dtype_1 = "VARCHAR(20)"
dtype_2 = "DATETIME(6)"
table_2 = "Linear_Actuator_Data"
entry_3 = "`Position`"
entry_4 = "Position_Cmd"
entry_5 = "Position_Error"
entry_6 = "`Timestamp`"
dtype_3 = "VARCHAR(5)"
dtype_4 = "VARCHAR(5)"
dtype_5 = "VARCHAR(5)"
dtype_6 = "DATETIME(6)"
drop_database_query_1 = "DROP TABLE IF EXISTS {}".format(table_1)
drop_database_query_2 = "DROP TABLE IF EXISTS {}".format(table_2)
dbc.execute(drop_database_query_1)
db.commit()
dbc.execute(drop_database_query_2)
db.commit()

Q1 = f"CREATE TABLE {table_1} ({entry_1} {dtype_1} NOT NULL, {entry_2} {dtype_2} NOT NULL)"
Q2 = f"CREATE TABLE {table_2} ({entry_3} {dtype_3} NOT NULL, {entry_4} {dtype_4} NOT NULL, {entry_5} {dtype_5} NOT NULL, {entry_6} {dtype_6} NOT NULL)"
dbc.execute(Q1)
dbc.execute(Q2)

In [ ]:
#Linear Actuator class
class LinearActuator(tk.Frame):
    def __init__(self, master):
        super().__init__(master)
        self.master = master
        self.master.title("UltraMotion Linear Actuator")
        self.serial_lock = threading.Lock()
        self.read_queue_UM = deque()
        self.write_queue = queue.Queue()
        self.data_length = 3
        self.buffer_UM = b''
        self.linear_queue = queue.Queue()
        self.motion_active = False
        self.position = 0

        self.recording = False


        #Read data
        self.read_frame = tk.LabelFrame(self.master, text="Data From Linear Actuator")
        self.read_frame.pack(fill="both", expand="yes", padx=20, pady=10)

        #Lin actuator widgets
        self.port_label_UM = tk.Label(self.read_frame, text="UM Port:")
        self.port_label_UM.grid(row=0, column=0, sticky="w")
        self.port_entry_UM = tk.Entry(self.read_frame, width=10)
        self.port_entry_UM.insert(0, "COM4")
        self.port_entry_UM.grid(row=0, column=1, padx=5, pady=5)
        self.baud_label_UM = tk.Label(self.read_frame, text="UM Baudrate (115200 Default):")
        self.baud_label_UM.grid(row=0, column=2, sticky="w")
        self.baud_entry_UM = tk.Entry(self.read_frame, width=10)
        self.baud_entry_UM.insert(0, "115200")
        self.baud_entry_UM.grid(row=0, column=3, padx=5, pady=5)

        #To Database
        self.record_frame = tk.LabelFrame(self.master, text="Record")
        self.record_frame.pack(fill="both", expand="yes", padx=20, pady=10)
        self.db_button = tk.Button(self.record_frame, text="Record to Database", command=self.toggle_database_recording)
        self.db_button.grid(row=0,column=1, padx=5, pady=5)

        #Motion
        self.motion_frame = tk.LabelFrame(self.master, text="Motion")
        self.motion_frame.pack(fill="both", expand="yes", padx=20, pady=10)
        self.linear_motion_button = tk.Button(self.motion_frame, text="Toggle Linear Motion", command=self.toggle_linear_motion)
        self.linear_motion_button.grid(row=0,column=0, padx=5, pady=5)

        #writing
        self.write_frame = tk.LabelFrame(self.master, text="Manual Adjustments")
        self.write_frame.pack(fill="both", expand="yes", padx=20, pady=10)

        #writing widgets
        self.write_entry = tk.Entry(self.write_frame, width=40)
        self.write_entry.grid(row=0, column=0, padx=5, pady=5)
        self.send_button = tk.Button(self.write_frame, text="Send", command=self.send_data)
        self.send_button.grid(row=0, column=1, padx=5, pady=5)
        self.instructions1= tk.Label(self.write_frame, text="Positional Movement: TA#, TE, TR")
        self.instructions1.grid(row=1, column=0)
        self.instructions2= tk.Label(self.write_frame, text="Position range (#): 3500(TR) - 18000(TE)")
        self.instructions2.grid(row=1, column=1)
        self.gottagofast= tk.Label(self.write_frame, text="Speed: SP")
        self.gottagofast.grid(row=2, column=0)
        self.FastAFBoi= tk.Label(self.write_frame, text="Speed Range: 200 000 default")
        self.FastAFBoi.grid(row=2, column=1)

        #connect
        self.connect_button = tk.Button(self.read_frame, text="Connect", command=self.connect_serial)
        self.connect_button.grid(row=0, column=4, padx=5, pady=5)
        self.disconnect_button = tk.Button(self.read_frame, text="Close Application", command=self.disconnect_close)
        self.disconnect_button.grid(row=0, column=5, padx=5, pady=5)

        #Text Box
        self.read_text = tk.Text(self.read_frame, width=50, height=10)
        self.read_text.grid(row=2, column=0, columnspan=20, padx=5, pady=5)

        #Read Values from config
        self.value1_label = tk.Label(self.read_frame, text="Value 1: ")
        self.value1_label.grid(row=3, column=0, padx=5, pady=5)
        self.value2_label = tk.Label(self.read_frame, text="Value 2: ")
        self.value2_label.grid(row=3, column=1, padx=5, pady=5)
        self.value3_label = tk.Label(self.read_frame, text="Value 3: ")
        self.value3_label.grid(row=3, column=2, padx=5, pady=5)

        self.read_thread_UM = None #Declare separate read thread, writing will be done in Main thread
        self.linear_thread = None

    def connect_serial(self):
        # Ports and baud, default values already entered (for windows)
        port_UM = self.port_entry_UM.get()
        baudrate_UM = self.baud_entry_UM.get()
        if port_UM == "":
            messagebox.showerror("Error", "Please enter a valid port.")
            return
        if baudrate_UM == "":
            messagebox.showerror("Error", "Please enter a valid baudrate.")
            return
        try:
            self.UM = serial.Serial(port_UM, int(baudrate_UM), timeout=1)
        except serial.SerialException:
            messagebox.showerror("Error", "Could not connect to serial port.")
            return
        self.connect_button.configure(state="disabled")

        #Initialization for lin actuator - XG to enable reading
        self.UM.write('XG\r'.encode())
        self.UM.write("SP50000\r".encode())

        #clears initial entries
        self.UM.read(self.UM.in_waiting)

        #Establish separate threads
        self.read_thread_UM = threading.Thread(target=self.read_data_UM)
        self.read_thread_UM.daemon = True
        self.read_thread_UM.start()

        self.linear_thread = threading.Thread(target=self.linear_motion)
        self.linear_thread.daemon = True
        self.linear_thread.start()

        #Call update UI function
        self.update_ui()

    def disconnect_close(self):
        try:
            self.UM.close()
        except Exception as e:
            print(f"Error closing serial port {self.UM}: {e}")
        self.connect_button.configure(state="normal")
        self.disconnect_button.configure(state="disabled")
        self.master.destroy()

    def read_data_UM(self):
        while self.UM.isOpen():
            self.serial_lock.acquire()
            if self.UM.in_waiting:
                self.buffer_UM += self.UM.read(self.UM.in_waiting)
                current_time = datetime.datetime.now()
                lines = self.buffer_UM.split(b'\r\n')
                # If the buffer ends with an incomplete line, keep it for the next iteration
                if not self.buffer_UM.endswith(b'\r\n'):
                    self.buffer_UM = lines[-1]
                    lines = lines[:-1]
                else:
                    self.buffer_UM = b''

                for line in lines:
                    if not line.strip():
                        continue
                    values = line.strip().decode().split(",")

                    if len(values) == 3:
                        values[0] = re.sub(r'\D', '', values[0]) # ensures data coming in is ipurely numerical
                        self.position = int(values[0])
                        packet_size = len(self.buffer_UM)/len(lines)
                        time_difference = datetime.timedelta(seconds=(packet_size / 115200))
                        measurement_time = current_time + time_difference
                        #print((values[0],values[1],values[2]), measurement_time)
                        self.read_queue_UM.append(((values[0],values[1],values[2]), measurement_time))

                    else:
                        continue

            self.serial_lock.release()

    def toggle_linear_motion(self):
        self.motion_active = not self.motion_active
        if self.motion_active:
            self.linear_motion_button.config(text="Stop Motion")
        else:
            self.linear_motion_button.config(text="Toggle Linear Motion")

    def linear_motion(self):
        while self.UM.isOpen():
            if self.motion_active == True:
                self.serial_lock.acquire()
                if self.position == 3500:
                    self.linear_queue.put("TE\r".encode())
                    print('TR reached, attempting to move to TE')
                elif self.position == 18000:
                    self.linear_queue.put("TR\r".encode())
                    print('TE reached, attempting to move to TR')

                try:
                    data = self.linear_queue.get(block=False, timeout=0.1)
                    self.UM.write(data)
                    self.linear_queue.empty()
                    self.UM.flush()

                except queue.Empty:
                    #print("gay frogs")
                    pass

                self.serial_lock.release()
            else:
                pass
            time.sleep(0.001)

    def toggle_database_recording(self):
        self.recording = not self.recording
        if self.recording:
            self.db_button.config(text="Stop Recording")
        else:
            self.db_button.config(text="Resume Recording")


    def send_data(self):
        data = self.write_entry.get()
        data = data + "\r"
        self.serial_lock.acquire()
        if data:
            data_bytes = data.encode()
            self.write_queue.put(data_bytes)
            self.write_entry.delete(0, tk.END)

        try:
            data = self.write_queue.get(block=True, timeout=0.1)
            print(f'From write queue: {data}')
            self.UM.write(data)
            self.UM.flush()

        except queue.Empty:
            pass

        finally:
            self.serial_lock.release()

    def update_ui(self):
        data_UM = []
        for i in range(50):
            try:
                if self.read_queue_UM:
                    values, time_UM = self.read_queue_UM[-1]
                    self.read_text.insert(tk.END, time_UM.strftime('%M:%S.%f') + "\n")
                    self.read_text.see(tk.END)

                    self.value1_label.config(text="Position: {}".format(values[0]))
                    self.value2_label.config(text="Position Command Setpoint: {}".format(values[1]))
                    self.value3_label.config(text="Position Error: {}".format(values[2]))

                    self.read_queue_UM.clear()

                    data_UM.append((values[0], values[1], values[2], time_UM))


                else:
                    continue

            except:
                pass

        if self.recording:
            query_UM = 'INSERT INTO {} ({}, {}, {}, {}) VALUES (%s, %s, %s, %s)'.format(table_2, entry_3, entry_4, entry_5, entry_6)
            dbc.executemany(query_UM, data_UM)
            db.commit()

        self.master.after(10, self.update_ui)




In [ ]:
#LV
class ForceSensor(tk.Frame):
    def __init__(self, master):
        super().__init__(master)
        self.master = master
        self.master.title("Force Sensor")
        self.serial_lock = threading.Lock()
        self.read_queue_LV = deque()
        self.buffer_LV = b''

        # read data
        self.read_frame = tk.LabelFrame(self.master, text="Read Data")
        self.read_frame.pack(fill="both", expand="yes", padx=20, pady=10)

        #force sensor widgets.
        self.port_label_LV = tk.Label(self.read_frame, text="LV Port:")
        self.port_label_LV.grid(row=0, column=0, sticky="w")
        self.port_entry_LV = tk.Entry(self.read_frame, width=10)
        self.port_entry_LV.insert(0, "COM3")
        self.port_entry_LV.grid(row=0, column=1, padx=5, pady=5)
        self.baud_label_LV = tk.Label(self.read_frame, text="LV Baudrate (230400 Default):")
        self.baud_label_LV.grid(row=0, column=2, sticky="w")
        self.baud_entry_LV = tk.Entry(self.read_frame, width=10)
        self.baud_entry_LV.insert(0, 230400)
        self.baud_entry_LV.grid(row=0, column=3, padx=5, pady=5)

        #connect
        self.connect_button = tk.Button(self.read_frame, text="Connect", command=self.connect_serial)
        self.connect_button.grid(row=0, column=4, padx=5, pady=5)

        self.disconnect_button = tk.Button(self.read_frame, text="Close Application", command=self.disconnect_close)
        self.disconnect_button.grid(row=0, column=5, padx=5, pady=5)

        #Text Box
        self.read_text = tk.Text(self.read_frame, width=50, height=10)
        self.read_text.grid(row=1, column=0, columnspan=20, padx=5, pady=5)

        #Read Values from config
        self.value4_label = tk.Label(self.read_frame, text="Force (N): ")
        self.value4_label.grid(row=2, column=0, padx=5, pady=5)
        self.value5_label = tk.Label(self.read_frame, text="Timestamp: ")
        self.value5_label.grid(row=3, column=0, padx=5, pady=5)

        self.read_thread_LV = None #Declare read thread for LV

    def connect_serial(self):
        # Ports and baud, default values already entered (for windows)
        port_LV = self.port_entry_LV.get()
        baudrate_LV = self.baud_entry_LV.get()
        if port_LV == "":
            messagebox.showerror("Error", "Please enter a valid port.")
            return
        if baudrate_LV == "":
            messagebox.showerror("Error", "Please enter a valid baudrate.")
            return
        try:
            self.LV = serial.Serial(port_LV, int(baudrate_LV), timeout=1)
        except serial.SerialException:
            messagebox.showerror("Error", "Could not connect to serial port.")
            return
        self.connect_button.configure(state="disabled")

        while True:
            self.LV.write(b'\r')
            time.sleep(0.1)
            response = self.LV.read(1)
            if response and response.decode() == 'A':
                print("Received Affirmative. Stopping loop.") #Affirmative - sensor responsive
                break

        self.LV.write(b'TARE\r') # zeroes sensor
        time.sleep(0.05)
        self.LV.write(b'UNITS N\r') # units to newtons
        time.sleep(0.05)
        self.LV.write(b'wc\r') #constant data streaming
        time.sleep(0.05)

        #clears initial entries
        self.LV.read(self.LV.in_waiting)

        #Establish separate threads
        self.read_thread_LV = threading.Thread(target=self.read_data_LV)
        self.read_thread_LV.daemon = True
        self.read_thread_LV.start()

        #Call update UI function
        self.update_ui()

    def disconnect_close(self):
        try:
            self.LV.close()
        except Exception as e:
            print(f"Error closing serial port {self.LV}: {e}")
        self.connect_button.configure(state="normal")
        self.disconnect_button.configure(state="disabled")
        self.master.destroy()

    def read_data_LV(self):
        while self.LV.isOpen():
            self.serial_lock.acquire()
            if self.LV.in_waiting:
                self.buffer_LV += self.LV.read(self.LV.in_waiting)
                lines = self.buffer_LV.split(b'\r\n')
                # If the buffer ends with an incomplete line, keep it for the next iteration
                if not self.buffer_LV.endswith(b'\r\n'):
                    self.buffer_LV = lines[-1]
                    lines = lines[:-1]
                else:
                    self.buffer_LV = b''

                measurement_time = datetime.datetime.now()

                for line in lines:
                    if not line.strip():
                        continue
                    measurement = line.strip().decode()
                    packet_size = len(self.buffer_LV)/len(lines)
                    time_difference = datetime.timedelta(seconds=(packet_size/230400))
                    measurement_time += time_difference

                    self.read_queue_LV.append((measurement, measurement_time))
            self.serial_lock.release()

    def update_ui(self):
        data_LV = []
        for i in range(25):
            try:
                if self.read_queue_LV:

                    force, time_LV = self.read_queue_LV[-1]
                    self.read_text.insert(tk.END, time_LV.strftime('%M:%S.%f') + "\n")
                    self.read_text.see(tk.END)
                    self.value4_label.config(text="Force: {}".format(force))
                    self.value5_label.config(text="Timestamp: {}".format(datetime.datetime.now()))
                    self.read_queue_LV.clear()
                    data_LV.append((force, time_LV))

                else:
                    continue

            except:
                pass


        if app1_instance.recording:
            query_LV = 'INSERT INTO {} ({}, {}) VALUES (%s, %s)'.format(table_1, entry_1, entry_2)
            dbc.executemany(query_LV, data_LV)
            db.commit()

        self.master.after(10, self.update_ui)




In [ ]:
#run program
if __name__ == "__main__":
    app1 = tk.Tk()
    app1_instance = LinearActuator(app1)

    app2 = tk.Tk()
    app2_instance = ForceSensor(app2)

    app1.mainloop()

In [ ]:
server = mysql.connector.connect(
    host='127.0.0.1', #local host
    user='root',    # root user: has access to everything
    passwd='ICELABMR',
)
server_cursor = server.cursor()
db = mysql.connector.connect(
    host='127.0.0.1',
    user='root',
    passwd='ICELABMR',
    database=database_name #connect to newly created database
)
dbc = db.cursor()

In [ ]:
def central_difference(x, t):
    n = len(x)
    dx_dt = np.zeros(n)
    for i in range(1, n-1):
        dx = x[i+1] - x[i-1]
        dt = (t[i+1] - t[i-1]).total_seconds()
        dx_dt[i] = dx / dt
    dt0 = (t[1] - t[0]).total_seconds()
    dx_dt[0] = (x[1] - x[0]) / dt0
    dt_n = (t[n-1] - t[n-2]).total_seconds()
    dx_dt[n-1] = (x[n-1] - x[n-2]) / dt_n
    return dx_dt

In [ ]:
dbc.execute("SELECT * FROM Linear_Actuator_Data")
result = dbc.fetchall()
columns = [column[0] for column in dbc.description]
UM = pd.DataFrame(result, columns=columns)
UM['Timestamp'] = pd.to_datetime(UM['Timestamp'])
UM.set_index('Timestamp', inplace=True)
UMnp = UM['Position'].values
UMnp = (UMnp.astype(float)-3500)*-0.074/(18000-3500)
plt.plot(UM.index, UMnp)
plt.plot(UM.index, central_difference(UMnp, UM.index))

In [ ]:
dbc.execute("SELECT * FROM Force_Data")
force_data = dbc.fetchall()
columns = [column[0] for column in dbc.description]
LV = pd.DataFrame(force_data, columns=columns)
LV['Timestamp'] = pd.to_datetime(LV['Timestamp'])
LV.set_index('Timestamp', inplace=True)
LV.index = pd.DatetimeIndex(UM.index).asi8
LVnp = LV['Force'].values
LVnp = LVnp.astype(float)
plt.plot(LV.index, LVnp)

In [ ]:
UM.to_csv('SP200000-AC10-20-50-motion.csv')
LV.to_csv('SP200000-AC10-20-50-force.csv')

In [ ]:
#Test read for INA 219
class INAsensor(tk.Frame):
    def __init__(self, master):
        super().__init__(master)
        self.master = master
        self.master.title("Current/Voltage Sensor")
        self.serial_lock = threading.Lock()
        self.read_queue_INA = deque()
        self.buffer_INA = b''

        # read data
        self.read_frame = tk.LabelFrame(self.master, text="Read Data")
        self.read_frame.pack(fill="both", expand="yes", padx=20, pady=10)

        #sensor widgets.
        self.port_label_INA = tk.Label(self.read_frame, text="INA219 Sensor Port:")
        self.port_label_INA.grid(row=0, column=0, sticky="w")
        self.port_entry_INA = tk.Entry(self.read_frame, width=10)
        self.port_entry_INA.insert(0, "COM3")
        self.port_entry_INA.grid(row=0, column=1, padx=5, pady=5)
        self.baud_label_INA = tk.Label(self.read_frame, text="INA219 Baudrate (115200 Default):")
        self.baud_label_INA.grid(row=0, column=2, sticky="w")
        self.baud_entry_INA = tk.Entry(self.read_frame, width=10)
        self.baud_entry_INA.insert(0, 115200)
        self.baud_entry_INA.grid(row=0, column=3, padx=5, pady=5)

        #connect
        self.connect_button = tk.Button(self.read_frame, text="Connect", command=self.connect_serial)
        self.connect_button.grid(row=0, column=4, padx=5, pady=5)

        self.disconnect_button = tk.Button(self.read_frame, text="Close Application", command=self.disconnect_close)
        self.disconnect_button.grid(row=0, column=5, padx=5, pady=5)

        #Text Box
        self.read_text = tk.Text(self.read_frame, width=50, height=10)
        self.read_text.grid(row=1, column=0, columnspan=20, padx=5, pady=5)

        #Read Values from config
        self.SV_label = tk.Label(self.read_frame, text="Shunt Voltage (mV): ")
        self.SV_label.grid(row=2, column=0, padx=5, pady=5)
        self.BV_label = tk.Label(self.read_frame, text="Bus Voltage (V): ")
        self.BV_label.grid(row=2, column=1, padx=5, pady=5)
        self.current_label = tk.Label(self.read_frame, text="Current (mA): ")
        self.current_label.grid(row=2, column=2, padx=5, pady=5)
        self.power_label = tk.Label(self.read_frame, text="Power (mW): ")
        self.power_label.grid(row=2, column=3, padx=5, pady=5)
        #load voltage = bus voltage + shunt voltage /1000
        self.time_label = tk.Label(self.read_frame, text="Timestamp: ")
        self.time_label.grid(row=2, column=4, padx=5, pady=5)

        self.read_thread_INA = None #Declare read thread for INA sensor

    def connect_serial(self):
        # Ports and baud, default values already entered (for windows)
        port_INA = self.port_entry_INA.get()
        baudrate_INA = self.baud_entry_INA.get()
        if port_INA == "":
            messagebox.showerror("Error", "Please enter a valid port.")
            return
        if baudrate_INA == "":
            messagebox.showerror("Error", "Please enter a valid baudrate.")
            return
        try:
            self.INA = serial.Serial(port_INA, int(baudrate_INA), timeout=1)
        except serial.SerialException:
            messagebox.showerror("Error", "Could not connect to serial port.")
            return
        self.connect_button.configure(state="disabled")

        #clears initial entries
        self.INA.read(self.INA.in_waiting)

        #Establish separate threads
        self.read_thread_INA = threading.Thread(target=self.read_data_INA)
        self.read_thread_INA.daemon = True
        self.read_thread_INA.start()

        #Call update UI function
        self.update_ui()

    def disconnect_close(self):
        try:
            self.INA.close()
        except Exception as e:
            print(f"Error closing serial port {self.INA}: {e}")
        self.connect_button.configure(state="normal")
        self.disconnect_button.configure(state="disabled")
        self.master.destroy()

    def read_data_INA(self):
        while self.INA.isOpen():
            self.serial_lock.acquire()
            if self.INA.in_waiting:
                self.buffer_INA += self.INA.read(self.INA.in_waiting)
                lines = self.buffer_INA.split(b'\r\n')
                # If the buffer ends with an incomplete line, keep it for the next iteration
                if not self.buffer_INA.endswith(b'\r\n'):
                    self.buffer_INA = lines[-1]
                    lines = lines[:-1]
                else:
                    self.buffer_INA = b''

                measurement_time = datetime.datetime.now()


                for line in lines:
                    if not line.strip():
                        continue
                    values = line.strip().decode().split(",")

                    if len(values) == 4:
                        packet_size = len(self.buffer_INA)/len(lines)
                        time_difference = datetime.timedelta(seconds=(packet_size / 115200))
                        measurement_time = measurement_time + time_difference
                        self.read_queue_INA.append(((values[0],values[1],values[2],values[3]), measurement_time))

                    else:
                        continue

            self.serial_lock.release()

    def update_ui(self):
        data_INA = []
        for i in range(50):
            try:
                if self.read_queue_INA:

                    SV, BV, CMA, PMW, time = self.read_queue_INA[-1]
                    self.read_text.insert(tk.END, time.strftime('%M:%S.%f') + "\n")
                    self.read_text.see(tk.END)
                    self.SV_label.config(text="Shunt Voltage (mV): {}".format(SV))
                    self.BV_label.config(text="Bus Voltage (V): {}".format(BV))
                    self.current_label.config(text="Current (mA): {}".format(CMA))
                    self.power_label.config(text="Power (mW): {}".format(PMW))
                    self.time_label.config(text="Timestamp: {}".format(datetime.datetime.now()))
                    self.read_queue_INA.clear()
                    data_INA.append((SV, BV, CMA, PMW, time))

                else:
                    continue

            except:
                pass

        '''
        if app1_instance.recording:
            query_LV = 'INSERT INTO {} ({}, {}) VALUES (%s, %s)'.format(table_3, entry_1, entry_2)
            dbc.executemany(query_LV, data_LV)
            db.commit()
        '''
        self.master.after(10, self.update_ui)


app3 = tk.Tk()
app3_instance = INAsensor(app3)
app3.mainloop()

